### Введение
#### Цели проекта
Провести анализ логов веб-сервера для выявления:
   1. 10 самых активных IP-адресов. 
   2. Количества запросов по HTTP-методам.
   3. Количества запросов с кодом ответа 404.
   4. Общего объема данных ответов по дням.
#### Описание данных
Логи веб-сервера включают следующие столбцы:
- **ip**: IP-адрес клиента.
- **timestamp**: Временная метка запроса.
- **method**: HTTP-метод (GET, POST, PUT, DELETE).
- **url**: URL запроса.
- **response_code**: Код ответа сервера (например, 200, 404).
- **response_size**: Размер ответа в байтах.
#### Инструменты
PySpark, Jupyter Notebook, PyCharm

In [21]:
# Импорт библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as _sum, date_format
from pyspark.sql.functions import min as _min, max as _max, avg, stddev, round, col

In [3]:
# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("WebServerLogsAnalysis") \
    .getOrCreate()

In [4]:
# Загрузка данных из CSV-файла, который был предварительно сгенерирован.
file_path = "web_server_logs.csv"
logs_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [5]:
# Проверка структуры данных
logs_df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)



In [6]:
# Пример первых записей
logs_df.show(5, truncate=False)

+---------------+-------------------+------+--------------------------+-------------+-------------+
|ip             |timestamp          |method|url                       |response_code|response_size|
+---------------+-------------------+------+--------------------------+-------------+-------------+
|223.116.182.173|2024-07-17 09:44:50|PUT   |posts                     |404          |8788         |
|147.79.235.189 |2024-06-16 06:38:22|PUT   |posts                     |404          |3169         |
|74.93.217.180  |2024-04-01 07:04:17|DELETE|wp-content/wp-content/list|404          |1107         |
|120.221.45.62  |2024-06-25 20:17:16|POST  |explore                   |404          |1315         |
|95.100.217.26  |2024-08-01 15:09:02|PUT   |list/blog                 |301          |1198         |
+---------------+-------------------+------+--------------------------+-------------+-------------+
only showing top 5 rows



#### Задача 1: Топ-10 самых активных IP-адресов

In [7]:
# Группируем данные по IP-адресам, подсчитываем количество запросов и выводим 10 самых активных адресов.
top_ips = logs_df.groupBy("ip") \
    .agg(count("*").alias("request_count")) \
    .orderBy(col("request_count").desc()) \
    .limit(10)

In [8]:
print("Топ-10 активных IP-адресов:")
top_ips.show()

Топ-10 активных IP-адресов:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|   74.22.21.145|            2|
|  121.33.155.20|            2|
|  71.129.102.44|            1|
| 193.227.246.65|            1|
| 116.91.101.251|            1|
|217.143.101.201|            1|
|  6.188.213.236|            1|
|   165.8.68.160|            1|
| 117.78.123.112|            1|
|   2.46.232.233|            1|
+---------------+-------------+



Среди 10 самых активных IP-адресов ярко выраженного лидера нет, потому что практически все адреса отправили не больше 1-го запроса и только пара адресов (74.22.21.145 и 121.33.155.20) отправили хотя бы 2 запроса.

#### Задача 2: Количество запросов по HTTP-методам

In [9]:
#Группируем данные по столбцу "method" и подсчитываем количество запросов для каждого метода.
method_counts = logs_df.groupBy("method") \
    .agg(count("*").alias("method_count")) \
    .orderBy(col("method_count").desc())

In [10]:
print("Количество запросов по HTTP-методам:")
method_counts.show()

Количество запросов по HTTP-методам:
+------+------------+
|method|method_count|
+------+------------+
|   PUT|       25182|
|  POST|       25123|
|   GET|       24854|
|DELETE|       24841|
+------+------------+



Наиболее часто используются методы PUT и POST, количество их запросов примерно одинаковое: 25182 и 25123 соответственно.

#### Задача 3: Количество запросов с кодом ответа 404

In [11]:
# Фильтруем строки, где response_code равен 404, и подсчитываем количество таких строк.
error_404_count = logs_df.filter(col("response_code") == 404) \
    .agg(count("*").alias("404_count"))

In [12]:
print("Количество запросов с кодом 404:")
error_404_count.show()

Количество запросов с кодом 404:
+---------+
|404_count|
+---------+
|    24778|
+---------+



Обнаружено 24778 запросов с кодом ответа 404.

#### Задача 4: Сумма размеров ответов по дням

In [15]:
# Извлекаем дату из временной метки и группируем данные по дате, подсчитывая общий объем ответов.
logs_df_with_date = logs_df.withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))

response_size_by_date = logs_df_with_date.groupBy("date") \
    .agg(_sum("response_size").alias("total_response_size")) \
    .orderBy(col("date").asc())

In [16]:
print("Общий объем данных ответов по дням:")
response_size_by_date.show()

Общий объем данных ответов по дням:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1586209|
|2024-01-02|            1522854|
|2024-01-03|            1457914|
|2024-01-04|            1498359|
|2024-01-05|            1544727|
|2024-01-06|            1651367|
|2024-01-07|            1407101|
|2024-01-08|            1568120|
|2024-01-09|            1624450|
|2024-01-10|            1608041|
|2024-01-11|            1577735|
|2024-01-12|            1518721|
|2024-01-13|            1649294|
|2024-01-14|            1396481|
|2024-01-15|            1618075|
|2024-01-16|            1560489|
|2024-01-17|            1658609|
|2024-01-18|            1644657|
|2024-01-19|            1623925|
|2024-01-20|            1439035|
+----------+-------------------+
only showing top 20 rows



In [22]:
# Описательная статистика для объема данных по дням
response_stats = response_size_by_date.agg(
    _min(col("total_response_size")).alias("min_response_size"),
    _max(col("total_response_size")).alias("max_response_size"),
    round(avg(col("total_response_size")), 2).alias("mean_response_size"),
    round(stddev(col("total_response_size")), 2).alias("stddev_response_size")
)

print("Описательная статистика для общего объема данных (с округлением):")
response_stats.show()

Описательная статистика для общего объема данных (с округлением):
+-----------------+-----------------+------------------+--------------------+
|min_response_size|max_response_size|mean_response_size|stddev_response_size|
+-----------------+-----------------+------------------+--------------------+
|           952258|          1780659|        1527125.26|           100661.01|
+-----------------+-----------------+------------------+--------------------+



In [23]:
# Вывод наиболее и наименее активных дней
most_active_day = response_size_by_date.orderBy(col("total_response_size").desc()).limit(1)
least_active_day = response_size_by_date.orderBy(col("total_response_size").asc()).limit(1)

print("Наиболее активный день по объему данных:")
most_active_day.show()

print("Наименее активный день по объему данных:")
least_active_day.show()

Наиболее активный день по объему данных:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-10-09|            1780659|
+----------+-------------------+

Наименее активный день по объему данных:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-11-26|             952258|
+----------+-------------------+



Наибольший объем данных был передан 9 Октября 2024, общий объем: 1780659 байт (~1,78 МБ). В среднем передается около 1,5 МБ данных каждый день.

#### Итоговые выводы
1. **Топ-10 активных IP-адресов**:
    - Наиболее активные IP: 74.22.21.145 и 121.33.155.20, количество запросов: 1.
    - Остальные IP-адреса отправили по 1 запросу.

 2. **Количество запросов по HTTP-методам**:
    - Наиболее популярный метод: PUT, количество запросов: 25182.
    - Наименее популярный метод: DELETE, количество запросов: 24841.

 3. **Запросы с кодом ответа 404**:
    - Всего обнаружено 24778 запросов с кодом 404.

 4. **Объем данных по дням**:
    - В среднем передается около 1,5 МБ данных каждый день.
    - Наибольший объем данных был передан 09/10/2024, общий объем: 1780659 байт (~1,78 МБ).
    - Минимальный объем данных был передан 26/11/2024, общий объем: 952258 байт (~0,95 МБ).

Таким образом, проведенный анализ позволяет выявить основные характеристики активности пользователей на сервере.